In [1]:
using Flux, DiffEqFlux, Plots, DifferentialEquations, Random, Optim, Interact, CuArrays
Random.seed!(1)
plotlyjs() # optional backend for plotting

┌ Warning: Your CUDA installation does not provide the CUPTI library, CUDAnative.@code_sass will be unavailable
└ @ CUDAnative /home/julius/.julia/packages/CUDAnative/hfulr/src/CUDAnative.jl:160
┌ Warning: CuArrays.jl found cuda, but did not find libcudnn. Some functionality will not be available.
└ @ Flux /home/julius/.julia/packages/Flux/CjjeX/src/Flux.jl:48


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-3544883803617090682\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-8611572905458294452\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-3544883803617090682\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

Plots.PlotlyJSBackend()

# Neural Differential Equations in Julia
> Exploring the [Flux.jl](https://github.com/FluxML/Flux.jl) and [DiffEqFlux.jl](https://github.com/JuliaDiffEq/DiffEqFlux.jl) packages. 


## Warm-Up: Using Flux for Linear Regression

[Flux](https://julialang.org/blog/2018/12/ml-language-compiler/): "...typical frameworks are all-encompassing monoliths in hundreds of thousands of lines of C++, Flux is only a thousand lines of straightforward Julia code. Simply take one package for gradients (Zygote.jl), one package for GPU support (CuArrays.jl), sprinkle with some light convenience functions, bake for fifteen minutes and out pops a fully-featured ML stack."

**Problem:** Given data $(x_i,y_i)_{i=0}^m$ we want to approximately solve the problem 

$$ \min_{W,b} \sum_{i=0}^m | Wx_i+b - y_i |^2. $$

### Data

In [2]:
# data specification
samples = 30
noise_lvl = 0.2

# underlying (unkown) model;  Note: comment out '|> gpu' for execution on cpu only
Ŵ = rand(1,2) |> gpu
b̂ = rand(1) |> gpu
ξ = rand(1, samples) |> gpu

# affine linear mapping
aff(mat,vec) = (x -> mat*x .+ vec)

# create data
x = rand(2, samples) |> gpu
y = aff(Ŵ, b̂)(x) .+ noise_lvl.*ξ

# plot helper function (see https://docs.juliaplots.org/latest/) 
function plotting(model = nothing)
  x1 = x2 = range(0, 1; length=100) 
  p = scatter(x[1,:], x[2,:], vec(y), markersize = 2, label="data")
  if model != nothing
      plot!(x1, x2, (x1, x2) -> model(cu([x1,x2]))[], st=:surface, fα=0.5, colorbar_entry=false)
  end
  return p
end

# plot
plotting()

┌ Warning: Performing scalar operations on GPU arrays: This is very slow, consider disallowing these operations with `allowscalar(false)`
└ @ GPUArrays /home/julius/.julia/packages/GPUArrays/1wgPO/src/indexing.jl:16


<!DOCTYPE html>
 
 
 Plots.jl

### Model

In [3]:
# initial model 
W = rand(1,2) |> gpu
b = rand(1) |> gpu
model = aff(W,b)

# mean squared error loss 
MSE(y1, y2) = sum(abs2, y1 .- y2) 
loss(model) = MSE(model(x), y)

# print loss
printloss(model, i=0) = i%20==0 ? println("Step: $i Loss: $(loss(model))") : nothing
printloss(model)

# plot
plotting(model)

Step: 0 Loss: 36.817202701180314


<!DOCTYPE html>
 
 
 Plots.jl

### Gradient Descent

**Idea:** To improve the prediction we can take the gradient of the loss w.r.t. $W$ and $b$ and perform gradient descent.

In contrast to TensorFlow or PyTorch in Python this is possible without tracing the operations in advance (Julia is just-in-time compiled, the *computational graph* is Julia’s own syntax).

In [4]:
# gradient steps and learning rate
steps = 100
ν = 0.02

# gradient descent
for i=1:steps
  g = gradient(() -> loss(model), params(W, b))
  W .-= ν .* g[W]
  b .-= ν .* g[b]
  printloss(model, i)
end

# plot
plotting(model)

Step: 20 Loss: 0.26878422009605546
Step: 40 Loss: 0.11856947495012708
Step: 60 Loss: 0.11694477520335714
Step: 80 Loss: 0.11682404319096173
Step: 100 Loss: 0.11680836864113592


<!DOCTYPE html>
 
 
 Plots.jl

### Shortcut

Let us use predefined Flux functions!

In [5]:
# model and initial parameters
flux_model = Dense(2, 1) |> gpu
ps = Flux.params(flux_model)

# loss
printloss(flux_model)

Step: 0 Loss: 0.25853781137289666


In [6]:
# train
for i=1:steps
  Flux.train!((x,y) -> MSE(flux_model(x), y), ps, [(x,y)], ADAM(0.02))
  printloss(flux_model, i)
end

# plot
plotting(flux_model)

Step: 20 Loss: 0.20194502549479232
Step: 40 Loss: 0.20194502549479232
Step: 60 Loss: 0.20194502549479232
Step: 80 Loss: 0.20194502549479232
Step: 100 Loss: 0.20194502549479232


<!DOCTYPE html>
 
 
 Plots.jl

In [7]:
# compare the model parameters
println("Parameter of first model: W = $W, b = $b") 
println("Parameter of second model: W = $(flux_model.W), b = $(flux_model.b)") 

Parameter of first model: W = Float32[0.36133677 0.29995155], b = Float32[0.10507983]
Parameter of second model: W = Float32[0.2937699 0.49803552], b = Float32[0.04]


## Neural Differential Equations using DiffEqFlux

[DiffEqFlux](https://julialang.org/blog/2019/01/fluxdiffeq/): "Layers have traditionally been simple functions like matrix multiply, but in the spirit of differentiable programming people are increasingly experimenting with much more complex functions, such as ray tracers and physics engines. Turns out that differential equations solvers fit this framework, too."


**Problem:** Given data $(t_i, u(t_i))_{i=0}^m$ of the solution to an *unkown* ODE

$$ u'(t) = f(u), \quad u(t_0) = u_0 $$

**Goal:**  Train a neural network model $\mathcal{N}_\Phi$ (with learnable parameters $\Phi$) to approximately recover $f$, i.e. learn the underlying ODE from data.

**Idea:** Numerically solve the *neural* ODE 

$$ \tilde{u}_\Phi'(t) = \mathcal{N}_{\Phi}(\tilde{u}_\Phi), \quad \tilde{u}_\Phi(t_0) = u_0 $$

at times $(t_i)_{i=0}^t$ with a package that allows computing the gradient of the error 
$$\sum_{i=0}^m \big( \tilde{u}_\Phi(t_i)-u(t_i)\big)^2$$

w.r.t. to $\Phi$ in order to perform first-order optimization. 

### Underlying (Unkown) Dynamics

In [8]:
# initial condition 
u0 = Float32[2.0f0] 

# time horizon, number of samples, and uniformly distributed points 
tspan = (0.0f0,15f0)
datasize = 100
t = sort(tspan[1] .+ rand(Float32, datasize)*(tspan[2]-tspan[1]))

# noise and noise_lvl
noise_lvl = 0.1
ξ = rand(Float32, datasize)

# true du/dt
f(u,p,t) = 2*sin.(u)

# solution of the true ODE at time-points t and initial condition u0 with additional noise
# (we could also use the exact solution u(t)=2cot^{-1}(2^{-2t}cot(1)) but we want to explore DifferentialEquations.jl)
noisy_u(u0, t, ξ) = Array(solve(ODEProblem(f, u0, tspan), Tsit5(), saveat=t)) .+ noise_lvl.* ξ'
u = noisy_u(u0, t, ξ)

# plot helper function
function plotsol(u, û = nothing)
  p = scatter(t, vec(u), label="data")
  if û != nothing
    scatter!(t, vec(û), label="prediction") 
  end
  return p
end   

# plot solution
plotsol(u)

<!DOCTYPE html>
 
 
 Plots.jl

### Neural Network Model

In [9]:
# neural network model
model = Chain(Dense(1,50,relu), Dense(50,100,relu), Dense(100,1))

# ODE solver for the neural network model and initial model parameters
n_ode = NeuralODE(model, tspan, Tsit5(), saveat=t)
Φ = n_ode.p

# prediction for given initial condition
ũ(Φ) = n_ode(u0,Φ)

# plot of the data and the (untrained) neural ODE prediction
plotsol(u, ũ(Φ))

<!DOCTYPE html>
 
 
 Plots.jl

### Optimization

In [10]:
# loss 
loss(Φ) = MSE(ũ(Φ), u)

# callback for training
function callback(p, l) 
  println("Loss: $l")
  return false
end

# optimize with ADAM
res1 = DiffEqFlux.sciml_train(loss, Φ, ADAM(0.008), cb=callback, maxiters=300)

Loss: 131.08140861694827
Loss: 65714.4570699383
Loss: 247.93695710122938
Loss: 340.62364276040194
Loss: 662.089489261422
Loss: 787.8625415508113
Loss: 840.1282458318688
Loss: 857.6561483723947
Loss: 859.0035557833866
Loss: 845.0563623031256
Loss: 816.4397793154303
Loss: 771.0046576949334
Loss: 705.2216253768916
Loss: 606.9197721272094
Loss: 454.64453425099947
Loss: 209.2089659251352
Loss: 44.30914217697906
Loss: 326.1292088350874
Loss: 23.952475282159067
Loss: 95.1944462655586
Loss: 145.20665198015726
Loss: 163.06208484842125
Loss: 154.24329210845542
Loss: 124.69440604426246
Loss: 80.9772124081042
Loss: 34.7282889470702
Loss: 11.779643787903076
Loss: 45.77503761611267
Loss: 76.33492410467261
Loss: 42.75253669037258
Loss: 13.36998608052395
Loss: 13.049857815640147
Loss: 25.736756320589844
Loss: 36.748685336037546
Loss: 39.33220903585598
Loss: 32.931185314065345
Loss: 21.7118017540725
Loss: 11.315803279431805
Loss: 8.27749587484906
Loss: 15.081555355026557
Loss: 22.46851008990603
Loss: 2

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Minimizer: [1.53e-01, 1.26e-02, 7.15e-02,  ...]
    Minimum:   1.301281e+00

 * Found with
    Algorithm:     ADAM
    Initial Point: [1.90e-01, -6.93e-03, 1.26e-01,  ...]

 * Convergence measures
    |x - x'|               = NaN ≰ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 0.0e+00

 * Work counters
    Seconds run:   445  (vs limit Inf)
    Iterations:    300
    f(x) calls:    300
    ∇f(x) calls:   300


In [11]:
# plot
plotsol(u, ũ(res1.minimizer))

<!DOCTYPE html>
 
 
 Plots.jl

In [12]:
# optimize with LBFGS
res2 = DiffEqFlux.sciml_train(loss, res1.minimizer, LBFGS(), cb=callback)

Loss: 1.2945915015835932
Loss: 1.2934518615644968
Loss: 1.2852871545281594
Loss: 0.4286015717596255
Loss: 0.2703118272979938
Loss: 0.09557075273754957
Loss: 0.08921015030902318
Loss: 0.08591491787742318
Loss: 0.08533088738480223
Loss: 0.08530314326185084
Loss: 0.08524243708548346
Loss: 0.08524243708548346
Loss: 0.08524243708548346


 * Status: success

 * Candidate solution
    Minimizer: [1.64e-01, 1.07e-02, 7.81e-02,  ...]
    Minimum:   8.524244e-02

 * Found with
    Algorithm:     L-BFGS
    Initial Point: [1.53e-01, 1.26e-02, 7.15e-02,  ...]

 * Convergence measures
    |x - x'|               = 9.31e-10 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.26e-09 ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 1.61e-02 ≰ 1.0e-08

 * Work counters
    Seconds run:   164  (vs limit Inf)
    Iterations:    12
    f(x) calls:    126
    ∇f(x) calls:   126


In [13]:
# plot
plotsol(u, ũ(res2.minimizer))

<!DOCTYPE html>
 
 
 Plots.jl

### Extrapolate

In [14]:
# compare the neural diff. eq. solution to the groundtruth for different initial values
u0 = Float32[6.] # new initial condition
t = sort(tspan[1] .+ rand(Float32, datasize)*(tspan[2]-tspan[1])) # new time points
plotsol(noisy_u(u0, t, 0), NeuralODE(model, tspan, Tsit5(), saveat=t)(u0, res2.minimizer)) # plot

<!DOCTYPE html>
 
 
 Plots.jl

In [15]:
# compare the functions f and the neural network model directly 
du = range(1, 5; length=100)
plot(du, f(du, (), ()), label="f")
plot!(du, vec(n_ode.re(res2.minimizer)(du')), label="neural network")

<!DOCTYPE html>
 
 
 Plots.jl

Note that we can also continue our training for different initial conditions (if such data is available).